# Inference Question:
In this competition you are asked to predict the forest cover type (the predominant kind of tree cover) from strictly cartographic variables (as opposed to remotely sensed data). 

In [17]:
import pandas as pd 
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
import warnings
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
from sklearn.ensemble import  GradientBoostingClassifier
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC, LinearSVC 
from sklearn.neural_network import MLPClassifier

from sklearn import tree

from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
import json
import pickle
from IPython.display import Image

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

k_folds = 5
%matplotlib inline

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv" , header = None)

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
print("Train data shape: %s" % str(train.shape))

Train data shape: (15120, 56)


In [4]:
print("Test data shape: %s" % str(test.shape))

Test data shape: (565893, 55)


In [5]:
train.head()

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type7,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type15,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,1,2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
1,2,2590,56,2,212,-6,390,220,235,151,6225,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
2,3,2804,139,9,268,65,3180,234,238,135,6121,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
3,4,2785,155,18,242,118,3090,238,238,122,6211,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2
4,5,2595,45,2,153,-1,391,220,234,150,6172,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5


In [11]:
X = np.array(train)
Y = np.array(train["Cover_Type"].tolist())

In [12]:
shuffle = np.random.permutation(np.arange(X.shape[0]))
X, Y = X[shuffle], Y[shuffle]

print('data shape: ', X.shape)
print('label shape:', Y.shape)

# Set some variables to hold test, dev, and training data.
test_data, test_labels = X[13000:], Y[13000:]
dev_data, dev_labels = X[12000:13000], Y[12000:13000]
train_data, train_labels = X[:12000], Y[:12000]
train_df = train.iloc[shuffle,:].iloc[:12000 , :]

data shape:  (15120, 56)
label shape: (15120,)


# EDA

In [9]:
display(train_df.describe())

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type7,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type15,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
count,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.00000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.0,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.0,12000.000000,12000.000000,12000.000000,12000.00000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.0,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000
mean,7565.081667,2749.704167,156.726917,16.517750,228.112917,50.902750,1713.016417,212.643917,218.922500,135.119333,1509.364083,0.238333,0.032500,0.420917,0.30825,0.023250,0.041417,0.063000,0.056583,0.011083,0.043417,0.0,0.000083,0.000750,0.139583,0.026917,0.014833,0.031500,0.010500,0.0,0.007667,0.040917,0.004167,0.00275,0.009250,0.000833,0.022833,0.049667,0.017167,0.0,0.003583,0.000917,0.000667,0.086333,0.045583,0.022250,0.046750,0.042250,0.001417,0.006083,0.000500,0.002250,0.048833,0.044667,0.029750,4.008250
std,4366.191134,417.387141,110.246750,8.454652,209.628670,60.941767,1322.028252,30.650996,22.758306,45.871732,1097.585469,0.426082,0.177331,0.493727,0.46179,0.150703,0.199260,0.242973,0.231054,0.104697,0.203802,0.0,0.009129,0.027377,0.346569,0.161847,0.120891,0.174672,0.101934,0.0,0.087227,0.198105,0.064418,0.05237,0.095735,0.028857,0.149378,0.217264,0.129898,0.0,0.059756,0.030264,0.025812,0.280867,0.208588,0.147502,0.211112,0.201167,0.037614,0.077761,0.022356,0.047383,0.215528,0.206580,0.169904,2.001712
min,3.000000,1863.000000,0.000000,0.000000,0.000000,-146.000000,0.000000,0.000000,99.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,3802.750000,2375.000000,65.000000,10.000000,67.000000,5.000000,765.000000,196.000000,207.000000,106.000000,730.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
50%,7551.000000,2753.000000,125.000000,15.000000,180.000000,32.000000,1315.000000,220.000000,222.000000,138.000000,1250.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0

In [10]:
train_df.head()

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type7,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type15,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
4304,4305,2163,130,19,0,0,1380,247,226,100,914,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4
4396,4397,2359,82,14,382,165,924,239,214,103,1441,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4
6923,6924,2699,130,8,295,30,872,233,236,135,424,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
12315,12316,2320,74,35,256,30,658,236,152,16,573,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
7228,7229,2952,98,10,153,24,390,237,225,118,726,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1


In [13]:
def count_zero_for_a_column(train_df, column_name):
    count_obj = train_df.apply(lambda x: True if x[column_name] == 0 else False , axis=1)
    no_rows = len(count_obj[count_obj == True].index)
    print ( column_name , ' : ' , no_rows )
print ("\033[1m" , 'Count of zero rows by column :' , "\033[0;0m")
column_list = ['Elevation', 'Aspect', 'Slope','Horizontal_Distance_To_Hydrology','Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm','Horizontal_Distance_To_Fire_Points']
for column_name in column_list:
    count_zero_for_a_column(train_df, column_name)

 Count of zero rows by column : 
Elevation  :  0
Aspect  :  84
Slope  :  4
Horizontal_Distance_To_Hydrology  :  1282
Vertical_Distance_To_Hydrology  :  1520
Horizontal_Distance_To_Roadways  :  1
Hillshade_9am  :  1
Hillshade_Noon  :  0
Hillshade_3pm  :  75
Horizontal_Distance_To_Fire_Points  :  2


# Model Testing

## KNN

In [24]:
def Q1():
    
  knn = KNeighborsClassifier(n_neighbors=1)

  knn.fit(train_data, train_labels) #train on processed train data

  pred_knn  = knn.predict(dev_data) #predict on dev data

  accuracy_count_knn = 0

  for j in range(pred_knn.shape[0]):
    if pred_knn[j] == dev_labels[j]:
      accuracy_count_knn += 1
    else:
      continue

  print("KNN accuracy with no filtering: " + str (accuracy_count_knn/pred_knn.shape[0]))

### STUDENT END ###  
  
Q1()

KNN accuracy with no filtering: 0.859


## Bernoulli NB w/ Grid Search

In [14]:
def NB(alphas):

    # BernoulliNB training
    model_nb = BernoulliNB()
    model_nb.fit(train_data, train_labels)

    # use cros validation = 5 to validate the alphas
    model_cv = GridSearchCV(estimator=model_nb, param_grid=alphas, cv=5, scoring='accuracy')
    model_cv.fit(train_data, train_labels)

    # create the data frame
    params = [param['alpha'] for param in model_cv.cv_results_["params"]]
    mean_test_score = model_cv.cv_results_["mean_test_score"]
    std_test_score = model_cv.cv_results_["std_test_score"]
    df = pd.DataFrame(list(zip(params, mean_test_score,std_test_score)),
                   columns =['alpha', 'mean_test_score',"std_test_score"])
    print(df)
    
alphas = {'alpha': [1.0e-10, 0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0]}
NB(alphas)

          alpha  mean_test_score  std_test_score
0  1.000000e-10         0.599667        0.009858
1  1.000000e-04         0.599667        0.009858
2  1.000000e-03         0.599667        0.009858
3  1.000000e-02         0.599667        0.009858
4  1.000000e-01         0.599667        0.009858
5  5.000000e-01         0.599583        0.010003
6  1.000000e+00         0.599333        0.010582
7  2.000000e+00         0.598250        0.011345
8  1.000000e+01         0.595250        0.012903


## Gaussian Naive Bayes

In [23]:
def GNB():

  
  ####Train on GaussianNB with no var_smoothing####
  model_gnb = GaussianNB()
  model_gnb.fit(train_data,train_labels)

  sigmas = model_gnb.sigma_
  thetas = model_gnb.theta_
   
  ## predict on dev data
  
  pred_gnb = model_gnb.predict(dev_data)
  accuracy_count_gnb = 0
  
  
  ###generate accuracy
  for j in range(dev_labels.shape[0]):
    if pred_gnb[j] == dev_labels[j]:
      accuracy_count_gnb += 1
    else:
      continue

  print("GaussianNB accuracy w/ no modification: " + str(accuracy_count_gnb/pred_gnb.shape[0]))

  ###Train on gaussianNB with var_smoothing = .1###

  model_gnb_s = GaussianNB(var_smoothing=.1)
  model_gnb_s.fit(train_data,train_labels)
  
  sigmas2 = model_gnb_s.sigma_
  thetas2 = model_gnb_s.theta_
  
 
  ## predict on dev data

  pred_gnb_s = model_gnb_s.predict(dev_data)
  accuracy_count_gnb_s = 0
  
  
  ###generate accuracy
  for j in range(dev_labels.shape[0]):
    if pred_gnb_s[j] == dev_labels[j]:
      accuracy_count_gnb_s += 1
    else:
      continue

  print("GaussianNB accuracy w/ var smoothing @ .1: " + str(accuracy_count_gnb_s/pred_gnb_s.shape[0]))
  
  #train on bernoulliNB

  model_gnb_s2 = BernoulliNB()
  model_gnb_s2.fit(train_data,train_labels)

  pred_gnb_s2 = model_gnb_s2.predict(dev_data)
  accuracy_count_gnb_s2 = 0

  for j in range(dev_labels.shape[0]):
    if pred_gnb_s2[j] == dev_labels[j]:
      accuracy_count_gnb_s2 += 1
    else:
      continue

  print("Bernoulli NB performance: " + str(accuracy_count_gnb_s2/pred_gnb_s2.shape[0]))


### STUDENT END ###
    
GNB()

GaussianNB accuracy w/ no modification: 0.998
GaussianNB accuracy w/ var smoothing @ .1: 0.294
Bernoulli NB performance: 0.584
